In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as mtick

depression_file_path = r'F:\Raine\depression conditions.csv'
visit_files = [
    r'F:\Raine\all_visits_updated.csv',  
]

depression_data = pd.read_csv(depression_file_path, parse_dates=['CONDITION_START_DATE'], dayfirst=True)

depression_data.dropna(subset=['PERSON_ID', 'CONDITION_START_DATE'], inplace=True)

depression_data = depression_data[depression_data['CONDITION_START_DATE'] >= '2018-01-01']

depression_data['Month'] = depression_data['CONDITION_START_DATE'].dt.to_period('M')

visit_data = pd.concat([
    pd.read_csv(file, parse_dates=['VISIT_START_DATE'], dayfirst=True)
    for file in visit_files
])

visit_data.dropna(subset=['PERSON_ID', 'VISIT_START_DATE'], inplace=True)

visit_data = visit_data[visit_data['VISIT_START_DATE'] >= '2018-01-01']

visit_data['Month'] = visit_data['VISIT_START_DATE'].dt.to_period('M')

demo_data = visit_data[['PERSON_ID','RACE_CONCEPT_ID']].drop_duplicates(subset='PERSON_ID')

depression_data = pd.merge(
    depression_data,
    demo_data,
    on='PERSON_ID',
    how='left'  
)

def map_gender(g_id):
    """ Map GENDER_CONCEPT_ID to a label. """
    if g_id == 8507:
        return 'Male'
    elif g_id == 8532:
        return 'Female'

def map_race(r_id):
    """ Map RACE_CONCEPT_ID to a label. """
    if r_id == 8515:
        return 'Asian'
    elif r_id == 8516:
        return 'Black'
    elif r_id == 8527:
        return 'White'
    else:
        return 'Other'

def map_ethnicity(e_id):
    """ Map ETHNICITY_CONCEPT_ID to a label. """
    if e_id == 38003563:
        return 'Hispanic/Latino'
    else:
        return 'Not Hispanic/Latino'

def map_age(age):
    """ Convert numeric age into a category. """
    if pd.isna(age):
        return 'Unknown'
    age = float(age)
    if age < 13:
        return '<13'
    elif 13 <= age <= 17:
        return '13-17'
    elif 18 <= age <= 34:
        return '18-34'
    elif 35 <= age <= 65:
        return '35-65'
    else:
        return '>65'

depression_data['RaceCategory']   = depression_data['RACE_CONCEPT_ID'].apply(map_race)

visit_data['RaceCategory']   = visit_data['RACE_CONCEPT_ID'].apply(map_race)

def plot_prevalence_by_category(category_col, title_str, y_lim=None):
    """
    category_col: str - one of ['GenderCategory', 'RaceCategory', 'EthCategory', 'AgeCategory']
    title_str   : str - for the plot title
    y_lim       : tuple or None - y-axis limits if desired
    """

    monthly_depr_counts = (
        depression_data
        .dropna(subset=[category_col])  
        .groupby(['Month', category_col])['PERSON_ID']
        .nunique()
        .reset_index(name='DepressionCount')
    )

    monthly_depr_counts['Month'] = monthly_depr_counts['Month'].dt.to_timestamp()

    monthly_visitors = (
        visit_data
        .dropna(subset=[category_col])  
        .groupby(['Month', category_col])['PERSON_ID']
        .nunique()
        .reset_index(name='VisitorCount')
    )
    monthly_visitors['Month'] = monthly_visitors['Month'].dt.to_timestamp()

    combined = pd.merge(monthly_depr_counts,
                        monthly_visitors,
                        on=['Month', category_col],
                        how='outer')  

    combined['DepressionCount'] = combined['DepressionCount'].fillna(0)
    combined['VisitorCount']    = combined['VisitorCount'].fillna(0)

    combined['RelativePrevalence'] = 0
    mask = combined['VisitorCount'] > 0
    combined.loc[mask, 'RelativePrevalence'] = (
        combined.loc[mask, 'DepressionCount'] / combined.loc[mask, 'VisitorCount']
    )

    pivoted = combined.pivot(index='Month', columns=category_col, values='RelativePrevalence')

    plt.figure(figsize=(14, 7))
    sns.lineplot(data=pivoted, marker='o')
    plt.title(f'Relative Prevalence of Depression by {title_str}', fontsize=16, weight='bold')
    plt.xlabel('Month', fontsize=13)
    plt.ylabel('Diagnoses / Unique Visitors', fontsize=13)
    plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
    plt.xticks(rotation=45)
    if y_lim is not None:
        plt.ylim(y_lim)
    plt.legend(title=title_str, fontsize=11)
    plt.tight_layout()
    plt.show()

plot_prevalence_by_category(
    category_col='RaceCategory',
    title_str='Race',
    y_lim=(0.015, 0.06)  
)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

file_path = r'F:\Raine\depression conditions.csv'

data = pd.read_csv(file_path, parse_dates=['CONDITION_START_DATE'], dayfirst=True)

data = data.dropna(subset=['PERSON_ID', 'CONDITION_START_DATE'])

earliest_conditions = data.groupby('PERSON_ID', as_index=False).agg({'CONDITION_START_DATE': 'min'})

earliest_conditions = earliest_conditions[earliest_conditions['CONDITION_START_DATE'] >= '2018-01-01']

assert earliest_conditions['PERSON_ID'].duplicated().sum() == 0, "Duplicate PERSON_IDs found after grouping!"

earliest_conditions['Month'] = earliest_conditions['CONDITION_START_DATE'].dt.to_period('M')

monthly_counts = earliest_conditions.groupby('Month').size()

monthly_counts_df = monthly_counts.reset_index(name='Diagnosis Count')
monthly_counts_df['Month'] = monthly_counts_df['Month'].dt.to_timestamp()
print(monthly_counts_df.to_string())

sns.set(style="whitegrid", context="talk") 
plt.figure(figsize=(16, 8))

monthly_counts_df = monthly_counts_df[0:-1]

sns.lineplot(
    data=monthly_counts_df,
    x='Month',
    y='Diagnosis Count',
    marker='o',
    color='#007ACC',
    linewidth=2.5,
    label='Monthly Earliest Diagnoses'
)

mean_val = monthly_counts_df['Diagnosis Count'][0:24].mean()
std_err = monthly_counts_df['Diagnosis Count'].std() / (len(monthly_counts_df) ** 0.5)
ci_value = 1.96 * std_err 

plt.axhline(mean_val, color='#FF5733', linestyle='--', linewidth=2, label='Mean (2018-19)')
plt.fill_between(monthly_counts_df['Month'], mean_val - ci_value, mean_val + ci_value, 
                 color='#FFC300', alpha=0.2, label='95% CI (Relative to 2018-19 mean)')

plt.title('Monthly Earliest Diagnoses of Depression (From 2018)', fontsize=18, weight='bold', pad=20)
plt.xlabel('Month', fontsize=14, labelpad=15)
plt.ylabel('Number of Unique Patients Diagnosed', fontsize=14, labelpad=15)
plt.ylim(200, monthly_counts_df['Diagnosis Count'].max()*1.1)
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)
plt.legend(fontsize=12, frameon=False, loc='upper left')

sns.despine()
plt.show()

In [ ]:
import csv
import sqlite3
from datetime import datetime
from dateutil.relativedelta import relativedelta
import os

def get_person_info(conn, person_id):
    """
    Fetch GENDER_CONCEPT_ID, RACE_CONCEPT_ID, ETHNICITY_CONCEPT_ID,
    YEAR_OF_BIRTH, MONTH_OF_BIRTH from the PERSON table for a given person_id.
    Returns a dict with the columns or None if not found.
    """
    query = """
    SELECT
        GENDER_CONCEPT_ID,
        RACE_CONCEPT_ID,
        ETHNICITY_CONCEPT_ID,
        YEAR_OF_BIRTH,
        MONTH_OF_BIRTH
    FROM PERSON
    WHERE PERSON_ID = ?
    """

    cursor = conn.execute(query, (person_id,))
    row = cursor.fetchone()
    if row is None:
        return None

    return {
        'GENDER_CONCEPT_ID': row[0],
        'RACE_CONCEPT_ID': row[1],
        'ETHNICITY_CONCEPT_ID': row[2],
        'YEAR_OF_BIRTH': row[3],
        'MONTH_OF_BIRTH': row[4]
    }

def calculate_age(visit_date_str, year_of_birth, month_of_birth):
    """
    Compute age (in full years) at visit_date_str, given year and month of birth.
    We assume day of birth is the 1st if only year/month is known.
    """
    if not visit_date_str or not year_of_birth or not month_of_birth:
        return None

    try:
        visit_date = datetime.strptime(visit_date_str, "%Y-%m-%d")
    except ValueError:

        return None

    try:
        birth_date = datetime(int(year_of_birth), int(month_of_birth), 1)
    except ValueError:
        return None

    age_delta = relativedelta(visit_date, birth_date)
    return age_delta.years

def main():

    input_csv_path = r"F:\Raine\all_visits.csv"
    output_csv_path = r"F:\Raine\all_visits_updated.csv"  
    sqlite_path = r"F:\AllMontefiore2024August.sqlite"

    conn = sqlite3.connect(sqlite_path)

    with open(input_csv_path, 'r', newline='', encoding='utf-8') as infile, \
         open(output_csv_path, 'w', newline='', encoding='utf-8') as outfile:

        reader = csv.DictReader(infile)

        fieldnames = reader.fieldnames + [
            'GENDER_CONCEPT_ID',
            'RACE_CONCEPT_ID',
            'ETHNICITY_CONCEPT_ID',
            'AGE'
        ]
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        writer.writeheader()

        for row in reader:
            person_id_str = row.get("PERSON_ID", "")
            visit_date_str = row.get("VISIT_START_DATE", "")

            try:
                person_id = int(person_id_str)
            except ValueError:
                person_id = None

            if person_id is not None:

                pinfo = get_person_info(conn, person_id)
                if pinfo is not None:

                    row["GENDER_CONCEPT_ID"] = pinfo['GENDER_CONCEPT_ID']
                    row["RACE_CONCEPT_ID"] = pinfo['RACE_CONCEPT_ID']
                    row["ETHNICITY_CONCEPT_ID"] = pinfo['ETHNICITY_CONCEPT_ID']

                    year_of_birth = pinfo['YEAR_OF_BIRTH']
                    month_of_birth = pinfo['MONTH_OF_BIRTH']
                    age = calculate_age(visit_date_str, year_of_birth, month_of_birth)
                    row["AGE"] = age
                else:

                    row["GENDER_CONCEPT_ID"] = None
                    row["RACE_CONCEPT_ID"] = None
                    row["ETHNICITY_CONCEPT_ID"] = None
                    row["AGE"] = None
            else:

                row["GENDER_CONCEPT_ID"] = None
                row["RACE_CONCEPT_ID"] = None
                row["ETHNICITY_CONCEPT_ID"] = None
                row["AGE"] = None

            writer.writerow(row)

    conn.close()

if __name__ == "__main__":
    main()

In [ ]:
import csv
import sqlite3
from datetime import datetime
from dateutil.relativedelta import relativedelta
import os

CHUNK_SIZE = 999  

def calculate_age(visit_date_str, year_of_birth, month_of_birth):
    """
    Compute age (in full years) at visit_date_str, given year and month of birth.
    We assume day of birth is the 1st if only year/month is known.
    """
    if not visit_date_str or not year_of_birth or not month_of_birth:
        return None

    try:
        visit_date = datetime.strptime(visit_date_str, "%Y-%m-%d")
    except ValueError:

        return None

    try:
        birth_date = datetime(int(year_of_birth), int(month_of_birth), 1)
    except ValueError:
        return None

    age_delta = relativedelta(visit_date, birth_date)
    return age_delta.years

def fetch_person_data(conn, person_ids_batch):
    """
    Fetch rows from PERSON for a list of person_ids in one query.
    Returns a dict keyed by PERSON_ID, e.g.:
    {
        1234: {'GENDER_CONCEPT_ID': 8507, 'RACE_CONCEPT_ID': ..., ...},
        5678: {...},
        ...
    }
    """

    if not person_ids_batch:
        return {}

    placeholders = ",".join("?" for _ in person_ids_batch)
    query = f"""
        SELECT
            PERSON_ID,
            GENDER_CONCEPT_ID,
            RACE_CONCEPT_ID,
            ETHNICITY_CONCEPT_ID,
            YEAR_OF_BIRTH,
            MONTH_OF_BIRTH
        FROM PERSON
        WHERE PERSON_ID IN ({placeholders})
    """

    cursor = conn.execute(query, person_ids_batch)
    rows = cursor.fetchall()

    result_dict = {}
    for row in rows:
        person_id = row[0]
        result_dict[person_id] = {
            'GENDER_CONCEPT_ID': row[1],
            'RACE_CONCEPT_ID': row[2],
            'ETHNICITY_CONCEPT_ID': row[3],
            'YEAR_OF_BIRTH': row[4],
            'MONTH_OF_BIRTH': row[5]
        }

    return result_dict

def process_chunk(chunk, conn, writer, base_fieldnames):
    """
    - Extract unique PERSON_IDs from the chunk.
    - Fetch the corresponding records from PERSON in a single query.
    - For each row in the chunk, compute AGE, and write out to CSV.
    """

    person_ids = []
    for row in chunk:
        pid_str = row.get("PERSON_ID", "")
        try:
            pid = int(pid_str)
            person_ids.append(pid)
        except ValueError:

            continue

    person_data_dict = fetch_person_data(conn, person_ids)

    for row in chunk:
        pid_str = row.get("PERSON_ID", "")
        try:
            pid = int(pid_str)
        except ValueError:

            pid = None

        if pid and pid in person_data_dict:
            pinfo = person_data_dict[pid]
            row["GENDER_CONCEPT_ID"] = pinfo['GENDER_CONCEPT_ID']
            row["RACE_CONCEPT_ID"] = pinfo['RACE_CONCEPT_ID']
            row["ETHNICITY_CONCEPT_ID"] = pinfo['ETHNICITY_CONCEPT_ID']

            year_of_birth = pinfo['YEAR_OF_BIRTH']
            month_of_birth = pinfo['MONTH_OF_BIRTH']
            visit_date_str = row.get("VISIT_START_DATE", "")
            row["AGE"] = calculate_age(visit_date_str, year_of_birth, month_of_birth)
        else:

            row["GENDER_CONCEPT_ID"] = None
            row["RACE_CONCEPT_ID"] = None
            row["ETHNICITY_CONCEPT_ID"] = None
            row["AGE"] = None

        writer.writerow({fn: row.get(fn, None) for fn in base_fieldnames})

def main():
    input_csv_path = r"F:\Raine\all_visits.csv"
    output_csv_path = r"F:\Raine\all_visits_updated1.csv"
    sqlite_path = r"F:\AllMontefiore2024August.sqlite"

    conn = sqlite3.connect(sqlite_path)

    with open(input_csv_path, 'r', newline='', encoding='utf-8') as infile, \
         open(output_csv_path, 'w', newline='', encoding='utf-8') as outfile:

        reader = csv.DictReader(infile)

        base_fieldnames = reader.fieldnames + [
            'GENDER_CONCEPT_ID',
            'RACE_CONCEPT_ID',
            'ETHNICITY_CONCEPT_ID',
            'AGE'
        ]
        writer = csv.DictWriter(outfile, fieldnames=base_fieldnames)
        writer.writeheader()

        chunk = []
        for row in reader:
            chunk.append(row)

            if len(chunk) >= CHUNK_SIZE:
                process_chunk(chunk, conn, writer, base_fieldnames)
                chunk = []

        if chunk:
            process_chunk(chunk, conn, writer, base_fieldnames)

    c

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

depression_file_path = r'F:\Raine\depression conditions.csv'
visit_files = [
    r'F:\Roham\BC Pandemic (with Hasan)\telehealth.csv',
    r'F:\Roham\BC Pandemic (with Hasan)\outpatient.csv',
    r'F:\Roham\BC Pandemic (with Hasan)\office visit.csv',
    r'F:\Roham\BC Pandemic (with Hasan)\inpatient.csv'
]

data = pd.read_csv(depression_file_path, parse_dates=['CONDITION_START_DATE'], dayfirst=True)
data = data.dropna(subset=['PERSON_ID', 'CONDITION_START_DATE'])
data = data[data['CONDITION_START_DATE'] >= '2018-01-01']
data['Month'] = data['CONDITION_START_DATE'].dt.to_period('M')

monthly_depression_counts = data.groupby('Month')['PERSON_ID']
monthly_depression_df = monthly_depression_counts.reset_index(name='Depression Diagnoses')
monthly_depression_df['Month'] = monthly_depression_df['Month'].dt.to_timestamp()

visit_data = pd.concat([pd.read_csv(file, parse_dates=['VISIT_START_DATE'], dayfirst=True) for file in visit_files])
visit_data = visit_data.dropna(subset=['PERSON_ID', 'VISIT_START_DATE'])
visit_data = visit_data[visit_data['VISIT_START_DATE'] >= '2018-01-01']
visit_data['Month'] = visit_data['VISIT_START_DATE'].dt.to_period('M')

monthly_visitors = visit_data.groupby('Month')['PERSON_ID'].nunique()
monthly_visitors_df = monthly_visitors.reset_index(name='Unique Visitors')
monthly_visitors_df['Month'] = monthly_visitors_df['Month'].dt.to_timestamp()

combined_df = pd.merge(monthly_depression_df, monthly_visitors_df, on='Month')

combined_df['Relative Prevalence'] = combined_df['Depression Diagnoses'] / combined_df['Unique Visitors']

combined_df = combined_df[0:-1]

print(combined_df)

sns.set(style="whitegrid", context="talk")
plt.figure(figsize=(16, 8))

sns.lineplot(
    data=combined_df,
    x='Month',
    y='Relative Prevalence',
    marker='o',
    color='#007ACC',  
    linewidth=2.5,
    label='Relative Prevalence of Depression'
)

mean_val = combined_df['Relative Prevalence'][0:24].mean()
std_err = combined_df['Relative Prevalence'].std() / (len(combined_df) ** 0.5)
ci_value = 1.96 * std_err

plt.axhline(mean_val, color='#FF5733', linestyle='--', linewidth=2, label='Mean (2018-19)')
plt.fill_between(combined_df['Month'], mean_val - ci_value, mean_val + ci_value,
                 color='#FFC300', alpha=0.2, label='95% CI (Relative to 2018-19 mean)')

plt.title('Relative Prevalence of Depression Diagnoses (From 2018)', fontsize=18, weight='bold', pad=20)
plt.xlabel('Month', fontsize=14, labelpad=15)
plt.ylabel('Relative Prevalence (Diagnoses per Visitor)', fontsize=14, labelpad=15)
plt.ylim(0.0275, 0.05)
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)
plt.legend(fontsize=12, frameon=False, loc='upper left')

sns.despine()
plt.show()

In [ ]:
#prevalence relative to visits by month. 

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Paths to files
depression_file_path = r'F:\Raine\depression conditions.csv'
visit_files = [
    r'F:\Roham\BC Pandemic (with Hasan)\telehealth.csv',
    r'F:\Roham\BC Pandemic (with Hasan)\outpatient.csv',
    r'F:\Roham\BC Pandemic (with Hasan)\office visit.csv',
    r'F:\Roham\BC Pandemic (with Hasan)\inpatient.csv'
]

# Read and process depression data
data = pd.read_csv(depression_file_path, parse_dates=['CONDITION_START_DATE'], dayfirst=True)
data = data.dropna(subset=['PERSON_ID', 'CONDITION_START_DATE'])
data = data[data['CONDITION_START_DATE'] >= '2018-01-01']
data['Month'] = data['CONDITION_START_DATE'].dt.to_period('M')

# Get unique depression diagnoses per month
monthly_depression_counts = data.groupby('Month')['PERSON_ID'].nunique()
monthly_depression_df = monthly_depression_counts.reset_index(name='Depression Diagnoses')
monthly_depression_df['Month'] = monthly_depression_df['Month'].dt.to_timestamp()

# Combine visit data from all files
visit_data = pd.concat([pd.read_csv(file, parse_dates=['VISIT_START_DATE'], dayfirst=True) for file in visit_files])
visit_data = visit_data.dropna(subset=['PERSON_ID', 'VISIT_START_DATE'])
visit_data = visit_data[visit_data['VISIT_START_DATE'] >= '2018-01-01']
visit_data['Month'] = visit_data['VISIT_START_DATE'].dt.to_period('M')

# Get unique visitors per month
monthly_visitors = visit_data.groupby('Month')['PERSON_ID'].nunique()
monthly_visitors_df = monthly_visitors.reset_index(name='Unique Visitors')
monthly_visitors_df['Month'] = monthly_visitors_df['Month'].dt.to_timestamp()

# Merge depression data with visit data
combined_df = pd.merge(monthly_depression_df, monthly_visitors_df, on='Month')

# Calculate relative prevalence
combined_df['Relative Prevalence'] = combined_df['Depression Diagnoses'] / combined_df['Unique Visitors']

combined_df = combined_df[0:-1]

print(combined_df)

# Plot relative prevalence
sns.set(style="whitegrid", context="talk")
plt.figure(figsize=(16, 8))

sns.lineplot(
    data=combined_df,
    x='Month',
    y='Relative Prevalence',
    marker='o',
    color='#007ACC',  
    linewidth=2.5,
    label='Relative Prevalence of Depression'
)

# Calculate mean and 95% CI for relative prevalence
mean_val = combined_df['Relative Prevalence'][0:24].mean()
std_err = combined_df['Relative Prevalence'].std() / (len(combined_df) ** 0.5)
ci_value = 1.96 * std_err

plt.axhline(mean_val, color='#FF5733', linestyle='--', linewidth=2, label='Mean (2018-19)')
plt.fill_between(combined_df['Month'], mean_val - ci_value, mean_val + ci_value,
                 color='#FFC300', alpha=0.2, label='95% CI (Relative to 2018-19 mean)')

plt.title('Relative Prevalence of Depression Diagnoses (From 2018)', fontsize=18, weight='bold', pad=20)
plt.xlabel('Month', fontsize=14, labelpad=15)
plt.ylabel('Relative Prevalence (Diagnoses per Visitor)', fontsize=14, labelpad=15)
plt.ylim(0.0275, 0.05)
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)
plt.legend(fontsize=12, frameon=False, loc='upper left')

sns.despine()
plt.show()

In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math

db_path = r'F:\AllMontefiore2024August.sqlite'
depression_file_path = r'F:\Raine\depression conditions.csv'
visit_files = [
    r'F:\Roham\BC Pandemic (with Hasan)\telehealth.csv',
    r'F:\Roham\BC Pandemic (with Hasan)\outpatient.csv',
    r'F:\Roham\BC Pandemic (with Hasan)\office visit.csv',
    r'F:\Roham\BC Pandemic (with Hasan)\inpatient.csv'
]
demographic_category = 'GENDER'  
demographic_values = ['Male', 'Female']  

dep_data = pd.read_csv(depression_file_path, parse_dates=['CONDITION_START_DATE'], dayfirst=True)
dep_data.dropna(subset=['PERSON_ID', 'CONDITION_START_DATE'], inplace=True)
dep_data = dep_data[dep_data['CONDITION_START_DATE'] >= '2018-01-01']
dep_data['Month'] = dep_data['CONDITION_START_DATE'].dt.to_period('M')

visit_data_list = []
for file in visit_files:
    df = pd.read_csv(file, parse_dates=['VISIT_START_DATE'], dayfirst=True)
    df.dropna(subset=['PERSON_ID', 'VISIT_START_DATE'], inplace=True)
    df = df[df['VISIT_START_DATE'] >= '2018-01-01']
    df['Month'] = df['VISIT_START_DATE'].dt.to_period('M')
    visit_data_list.append(df)

visit_data = pd.concat(visit_data_list, ignore_index=True)

conn = sqlite3.connect(db_path)

if demographic_category == 'GENDER':
    concept_id_field = 'GENDER_CONCEPT_ID'
elif demographic_category == 'RACE':
    concept_id_field = 'RACE_CONCEPT_ID'
elif demographic_category == 'ETHNICITY':
    concept_id_field = 'ETHNICITY_CONCEPT_ID'
else:
    raise ValueError("Invalid demographic category. Must be 'GENDER', 'RACE', or 'ETHNICITY'.")

concept_query = f"""
    SELECT CONCEPT_ID, CONCEPT_NAME
    FROM Concept
    WHERE CONCEPT_ID IN (
        SELECT DISTINCT {concept_id_field}
        FROM Person
    )
"""
concepts_df = pd.read_sql(concept_query, conn)
id_to_category = dict(zip(concepts_df['CONCEPT_ID'], concepts_df['CONCEPT_NAME']))

person_query = f"""
    SELECT PERSON_ID, {concept_id_field}
    FROM Person
"""
person_demo_df = pd.read_sql(person_query, conn)
conn.close()

person_demo_df[demographic_category] = person_demo_df[concept_id_field].map(id_to_category)
demo_map = dict(zip(person_demo_df['PERSON_ID'], person_demo_df[demographic_category]))

dep_data[demographic_category] = dep_data['PERSON_ID'].map(demo_map)
visit_data[demographic_category] = visit_data['PERSON_ID'].map(demo_map)

dep_counts = (
    dep_data
    .groupby(['Month', demographic_category])['PERSON_ID']
    .nunique()  
    .reset_index(name='Unique_Dep_Visits')
)

visit_counts = (
    visit_data
    .groupby(['Month', demographic_category])['PERSON_ID']
    .nunique()  
    .reset_index(name='Unique_Total_Visits')
)

relprev_df = pd.merge(dep_counts, visit_counts, on=['Month', demographic_category], how='outer')

relprev_df['Relative Prevalence'] = (
    relprev_df['Unique_Dep_Visits'] / relprev_df['Unique_Total_Visits']
).fillna(0)

relprev_pivot = relprev_df.pivot(index='Month', columns=demographic_category, values='Relative Prevalence').reset_index()

for val in demographic_values:
    if val not in relprev_pivot.columns:
        relprev_pivot[val] = 0

relprev_pivot['Month'] = relprev_pivot['Month'].dt.to_timestamp()

sns.set(style="whitegrid", context="talk")
plt.figure(figsize=(16, 8))

for val in demographic_values:
    sns.lineplot(
        data=relprev_pivot,
        x='Month',
        y=val,
        marker='o',
        linewidth=2.5,
        label=f'Relative Prevalence ({val})'
    )

baseline = relprev_pivot[relprev_pivot['Month'] < '2020-01-01']

import numpy as np
for val in demographic_values:
    mean_val = baseline[val].mean() if not baseline.empty else 0
    std_err = baseline[val].std() / np.sqrt(len(baseline)) if not baseline.empty else 0
    ci_value = 1.96 * std_err
    plt.axhline(mean_val, linestyle='--', linewidth=2, label=f'{val} Mean (2018-19)')
    plt.fill_between(
        relprev_pivot['Month'],
        mean_val - ci_value,
        mean_val + ci_value,
        alpha=0.2,
        label=f'{val} 95% CI (Rel. to 2018-19)'
    )

plt.title(f'Relative Prevalence of Depression (Unique) by {demographic_category} (From 2018)', 
          fontsize=18, weight='bold', pad=20)
plt.xlabel('Month', fontsize=14, labelpad=15)
plt.ylabel('Relative Prevalence (Unique Dep Visits / Unique Total Visits)', fontsize=14, labelpad=15)
max_val = relprev_pivot[demographic_values].max().max()
plt.ylim(0, max_val * 1.1 if max_val > 0 else 1)
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)
plt.legend(fontsize=12, frameon=False, loc='upper left')
sns.despine()
plt.tight_layout()
plt.show()